In [ ]:
import os
import pathlib
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [ ]:
# PROJ_PATH = Path().cwd().parent
# DATA_PATH = PROJ_PATH.joinpath('data')
# In[ ]:
# ** loading path info **
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
curr_file_name = os.path.splitext(os.path.basename(os.path.abspath('')))[0]
data_dir = pathlib.Path('{}/data/eicu/'.format(parent_dir))
data_ps_dir = data_dir.joinpath('ps')
pathlib.Path.mkdir(data_dir, mode=0o777, parents=True, exist_ok=True)


In [ ]:
icu_list = ['MSICU', 'CTICU', 'SICU', 'CCUCTICU', 'MICU', 'NICU', 'CICU', 'CSICU']
icu_df_dict = {}
for icu in icu_list:
    icu_df_dict[icu] = pd.read_feather('{}/{}_df.feather'.format(data_ps_dir, icu))
    print('icu : ', icu, ' len(columns) : ', len(icu_df_dict[icu].columns), ' len(df) : ', len(icu_df_dict[icu]))
    

In [ ]:
def parsing(df):
    drop_list = ['apacheadmissiondx', 'hospitaladmittime24', 'hospitaldischargelocation', 'hospitaldischargetime24', 'patienthealthsystemstayid', 
                'uniquepid', 'unitadmittime24', 'unitdischargelocation', 'unitdischargestatus', 'unitdischargetime24', 'unittype']
    df.drop(columns = list(set(df.columns)&set(drop_list)), inplace=True)
    encoder = {}
    encoder['gender'] = {"M":1, "F":0, "Female":0, "Male":1}
    encoder['외국인여부'] = {"N":0, "Y":1, 'K':np.nan}
    encoder['death'] = {'Alive':0, "Expired":1}
    encoder['ethnicity'] = {"African American":0, "Caucasian":1,"Hispanic":2, "Asian":3, 'Native American':4, 'Other/Unknown':5}
    df_unitadmitsource = {col : idx for idx, col in enumerate(df.unitadmitsource.unique())}
    encoder['unitadmitsource'] = df_unitadmitsource
    df_unitstaytype = {col : idx for idx, col in enumerate(df.unitstaytype.unique())}
    encoder['unitstaytype'] = df_unitstaytype
    df_hospitaladmitsource = {col : idx for idx, col in enumerate(df.hospitaladmitsource.unique())}
    encoder['hospitaladmitsource'] = df_hospitaladmitsource
    encoder['hospitaldischargestatus'] = {"Expired":1, "Alive":0}
    encoder["death"] = {"Alive":0, "Expired":1}
    df = df.replace(encoder)
    df['age'] = df.age.str.replace('>','').astype('float')
    df['gender'] = df.gender.replace('Other|Unknown',np.nan, regex=True)
    # df = df.dropna(subset=['gender', 'age'], axis=0)
    df = df.dropna(axis=0)
    # df = df.rename(columns = {'hospitaldischargestatus':'death', "admissionheight":'height','admissionweight':'weight', 'meanbp':'bp'})
    # # df.head()
    # df = df.rename(columns = {'patientunitstayid':'연구등록번호'})
    return df

for icu in icu_list:
    print(icu)
    icu_df_dict[icu] = parsing(icu_df_dict[icu])
    print('icu : ', icu, ' len(columns) : ', len(icu_df_dict[icu].columns), ' len(df) : ', len(icu_df_dict[icu]))

In [ ]:
HICU = pd.read_feather(data_dir.joinpath('hicu.feather'))
HICU['bp'] = HICU.bp.str.extract('(\d{2,3})/(\d{2,3})').apply(lambda x : (float(x[0]) + 2*float(x[1]))/3, axis=1)
HICU['temperature'] = HICU.temperature.str.replace(',','.').str.extract("(\d{2}[.,\,]\d{1,2}|\d{2})").astype('float')
HICU['heartrate'] = HICU.heartrate.str.extract("(\d{2,3})")
HICU['respiratoryrate'] = HICU.respiratoryrate.str.extract("(\d{2})")
import numpy as np
encoder = {}
encoder['gender'] = {"M":1, "F":0, "Female":0, "Male":1}
encoder['외국인여부'] = {"N":0, "Y":1, 'K':np.nan}
encoder['death'] = {'Alive':0, "Expired":1}
HICU = HICU.replace(encoder)
HICU.외국인여부.unique()
HICU = HICU.astype({'age':'float32', 'height':"float", 'weight':"float", 'heartrate':"float", "respiratoryrate":"float", 
                                     "신장Z":'float', "신장P":"float", '체중Z':"float", "체중P":'float', '체표면적':"float",
                                     "death":'float'})
HICU['외국인여부'] = HICU.외국인여부.fillna(2)
death = HICU['death']
HICU = HICU.groupby('death').transform(lambda group : group.fillna(group.mean()))
HICU['death'] = death
HICU.to_feather(data_dir.joinpath('ps','HICU.feather'))

## 데이터 파씽 진행
- common feature는 비슷한 형식으로 encoding 진행
- specific feature에 대해서는 dummy encoding을 진행해야 한다.


Parsing with MICU, SICU

Free text로 되어 있는 admissiondx는 쓰지 않는다. in eICU

Common Feature, Specific Feature 추리기

In [ ]:
common_features = set(HICU.columns)
for icu in icu_list:
    common_features = common_features & set(icu_df_dict[icu].columns)
common_features.remove('death')
common_features.remove('연구등록번호')

In [ ]:
specific_features = {}
for icu in icu_list:
    specific_features[icu] = set(icu_df_dict[icu].columns) - common_features - set(['연구등록번호', 'death'])
    print(icu, 'specific : ', len(specific_features[icu]) )


In [ ]:
specific_features['HICU'] = set(HICU.columns) - common_features - set(['연구등록번호', 'death'])
    


In [ ]:
from sklearn.preprocessing import MinMaxScaler
def _min_max_scaler(df):
    min_max_scalar = MinMaxScaler()
    fitted = min_max_scalar.fit_transform(df[df.columns[1:]])
    df[df.columns[1:]] = fitted
    return df

for icu in icu_list:
    data_ps2_dir = pathlib.Path.joinpath(data_dir, 'ps2')
    pathlib.Path.mkdir(data_ps2_dir, mode=0o777, parents=True, exist_ok=True)
    icu_df_dict[icu].reset_index(drop=True, inplace=True)
    icu_df_dict[icu] = _min_max_scaler(icu_df_dict[icu])
    icu_df_dict[icu].to_feather('{}/{}_df.feather'.format(data_ps2_dir, icu))


In [ ]:
HICU.to_feather('{}/{}_df.feather'.format(data_ps2_dir, 'HICU'))

# Data EDA

## Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def _min_max_scaler(df):
    min_max_scalar = MinMaxScaler()
    fitted = min_max_scalar.fit_transform(df[df.columns[1:]])
    df[df.columns[1:]] = fitted
    return df

def collect_significant_variables(df, outcome, variables):
    X = df[variables]
    y = df[outcome]
    lasso = Lasso(alpha=0.003)
    # print(X)
    lasso.fit(X, y)
    print(lasso.coef_)
    importance = np.abs(lasso.coef_)
    selected_features = np.array(X.columns)[importance > 0]
    print(selected_features)
    print(len(selected_features))
    return selected_features

# for icu in icu_list:
#     icu_df_dict[icu] = _min_max_scaler(icu_df_dict[icu])

for icu in icu_list:
    icu_df_dict[icu] = pd.read_feather('{}/{}.feather'.format(data_dir, icu))


In [ ]:
icu_df_dict[icu]['hospitaldischargestatus']

In [ ]:
selected_features = {}    
for icu in icu_list:
    outcome = 'death'
    selected_features[icu] = collect_significant_variables(icu_df_dict[icu], outcome, specific_features[icu])
    with open(data_ps2_dir.joinpath('{}_selected.pkl'.format(icu)),'wb') as f:
        pickle.dump(selected_features[icu], f)
selected_features

In [ ]:
import yaml
with open(data_ps2_dir.joinpath('feature_book.yaml'),'w') as f:
    yaml.dump(selected_features, f, allow_unicode=True)

Get Specific variables for eICU datas

In [ ]:
def collect_significant_variables(df, outcome, variables):
    X = df[variables]
    y = df[outcome]
    lasso = Lasso(alpha=0.0010)
    # print(X)
    lasso.fit(X, y)
    print(lasso.coef_)
    importance = np.abs(lasso.coef_)
    selected_features = np.array(X.columns)[importance > 0]
    print(selected_features)
    print(len(selected_features))
    return selected_features

In [ ]:
concat_df = pd.DataFrame()
for icu in icu_list:
    concat_df = pd.concat([concat_df, icu_df_dict[icu]], axis=0)
feature_all = list(set(concat_df.columns) - set(['patientunitstayid', 'hospitaldischargestatus']))

In [ ]:
# concat_df['hospitaldischargestatus'].hist()
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x ='hospitaldischargestatus', data = concat_df)

In [ ]:
concat_df.hospitaldischargestatus.unique()

In [ ]:
feature_list = collect_significant_variables(concat_df, "hospitaldischargestatus", feature_all)